# Multiple linear regression

- the multiple linear regression looks lik: .y = b0 + b1*x1 + b2*x2 ....
- In multiple linear regression there is no need feature scaling. 


In [ ]:
# Import the libraries